This script aims to extract the case summary / case report tag from the NXMLs, in order to determine diagnosis similartiy based on generated queries

In [1]:
import json


In [69]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)
matching_uids_xml_paths[0:1]

[['/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/12886_2021_Article_2004.nxml',
  '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC5563556/5563556_1/TJO-47-243.nxml']]

In [76]:
from lxml import etree
from multiprocessing import Pool

# Create an XPath expression that matches <sec> elements with a <title> containing any of the keywords.
# Using translate() ensures a case-insensitive search.
xpath_expr = (
    "//sec["
    "  contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'case presentation') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'case report') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'case description') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'case summary') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'patient presentation') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'patient report') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'patient description') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'patient summary') "
    "or contains(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'case') "
    "or (normalize-space(translate(title, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')) = 'patient') "
    "]"
)


def process_file_list(file_list):
    """
    Given a list of file paths, parse each file, run the XPath,
    gather the text from <sec> paragraphs, return a dict { filepath: text }.
    """
    single_pair_case_summary = {}
    for file in file_list:
        tree = etree.parse(file)
        sections = tree.xpath(xpath_expr)
        for sec in sections:
            paragraphs = sec.xpath(".//p//text()")
            text = " ".join(paragraphs)
            single_pair_case_summary[file] = text
    return single_pair_case_summary

def multiprocess_case_summaries():
    with Pool() as pool:
        # Map each sub-list to a worker process
        case_summary_pairs_list = pool.map(process_file_list, matching_uids_xml_paths)

    return case_summary_pairs_list


case_summary_pairs_list = multiprocess_case_summaries()



In [77]:
case_summary_pairs_list = [list(case_summary_pairs_list[idx].values()) for idx in range(len(case_summary_pairs_list))]

In [78]:
def calc_indices_summaries_list(case_summary_pairs_list):
    """
    Returns two lists:
      - A list of indices corresponding to elements depicting both cases.
      - A list of indices corresponding to elements depicting less than 2 cases.
    """
    valid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) == 2]
    invalid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) != 2]

    return valid_indices, invalid_indices



In [79]:
len(calc_indices_summaries_list(case_summary_pairs_list)[1])

83

In [80]:
len(case_summary_pairs_list)

31692

In [81]:
v, i = calc_indices_summaries_list(case_summary_pairs_list)
len(v)

31609

In [ ]:
# with open('/cs/labs/tomhope/dhtandguy21/largeListsGuy/case_summary_pairs_list.json', 'w') as f:
#     json.dump(case_summary_pairs_list, f)

with open('/cs/labs/tomhope/dhtandguy21/largeListsGuy/case_summary_pairs_list.json', 'r') as f:
    case_summary_pairs_list = json.load(f)
case_summary_pairs_list[0]

['A 40-year-old woman who first presented with complaints of decreased vision and metamorphopsia in her right eye was diagnosed with CO 9\u2009years ago. At that time her best corrected visual acuity (BCVA) was 12/20. Both eyes had normal anterior segments. Right fundus examination showed a geographic-shaped, yellowish-white choroidal lesion surrounding the optic disc in the right eye (Fig.\xa0 1 ). B-scan ultrasonography of right eye revealed a typical dense echogenic plaque which causing acoustic shadowing behind (Fig.\xa0 2 ). FFA and ICGA had no evidence of CNV except early hyperfluorescent choroidal filling pattern with late diffuse staining. Computerized tomography (CT) showed a hyperdense choroidal plaque with the same density as bone (Fig.\xa0 3 ). Optical coherence tomography (OCT) demonstrated serous retinal detachments at the initial examination (Fig.\xa0 4 ).\n Fig. 1 Fundus photograph showed a yellowish white, peripapillary and sharply demarcated choroidal lesion involving

In [ ]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/imgs_paths.json', 'r') as f:
    imgs_paths = json.load(f)

In [ ]:
with open('/cs/labs/tomhope/dhtandguy21/largeListsGuy/imgs_paths_diagrams.json', 'r') as f:
    imgs_paths_diagrams = json.load(f)


In [ ]:
with open('/cs/labs/tomhope/dhtandguy21/largeListsGuy/pairs_text_list.json', 'r') as f:
    pairs_text_list = json.load(f)

with open('/cs/labs/tomhope/dhtandguy21/largeListsGuy/pairs_image_list.json', 'r') as f:
    pairs_image_list = json.load(f)


In [ ]:
pairs_image_list[0:1]

[[['/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_1.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_2.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_3.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_4.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_5.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_6.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_7.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_8.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC8167975/8167975_1/8167975_1_9.jpg'],
  ['/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2/PMC5563556/5563556_1/5563556_1_3.jpg',
   '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus

In [ ]:
from pathlib import Path

def remap_data_path(
    old_path: str,
    old_root: str = '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2',
    new_root: str = '/cs/labs/tomhope/dhtandguy21/restore_yuvalbus/data2'
) -> str:
    """
    Rebase an absolute path from old_root to new_root, keeping the same
    sub‐path under 'data2'.

    e.g. remap_data_path(
           '/cs/.../data2/PMC1131917/1131917_1/1131917_1_5.jpg'
         )
         returns
         '/cs/.../data2/PMC1131917/1131917_1/1131917_1_5.jpg'
    """
    p = Path(old_path)
    rel = p.relative_to(old_root)
    return str(Path(new_root) / rel)


In [ ]:
list_text = [
    [   # for each pair
        [   # for each path
            remap_data_path(text_path)
            for text_path in patient_list
        ]
        for patient_list in pair_text
    ]
    for pair_text in pairs_text_list
]

In [ ]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/pairs_text_list.json', 'w') as f:
    json.dump(list_text, f)

In [ ]:
list_images = [
    [   # for each pair
        [   # for each path
            remap_data_path(image_path)
            for image_path in patient_list
        ]
        for patient_list in pair_image
    ]
    for pair_image in pairs_image_list
]

In [ ]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/pairs_image_list.json', 'w') as f:
    json.dump(list_images, f)

In [63]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)
matching_uids_xml_paths[0:1]

[['/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC8167975/8167975_1/12886_2021_Article_2004.nxml',
  '/cs/labs/tomhope/yuvalbus/pmc/pythonProject/data2/PMC5563556/5563556_1/TJO-47-243.nxml']]

In [64]:
xml_paths = [
    [
        remap_data_path(xml_path)
        for xml_path in pair
    ]
    for pair in matching_uids_xml_paths
]

In [68]:
with open ('/cs/labs/tomhope/dhtandguy21/largeListsGuy/matching_uids_xml_paths.json', 'w') as f:
    json.dump(xml_paths, f)